### 5.2.1 FM模型

#### 一. 二阶特征组合
1. 为什么引入二阶特征组合   
 一般做CTR预估的方法, 最后一步是使用逻辑回归预测点击概率.  
 前篇的GBDT+LR的组合, 就是使用GBDT的树型结构, 模拟"苏格拉底式诘问"对特征进行非线性组合的结果.   
 除此之外, 我们想到, 如果任意组合2个特征, 形成二阶特征组合, 则会有更好表现. 此时, 加入二阶特征组合的LR为 : $$\hat { y } =b+{ w }_{ 1 }x+\sum _{ i=1 }^{ n }{ \sum _{ j=i+1 }^{ n }{ { w }_{ ij }{ x }_{ i }{ x }_{ j } }  } $$  
2. 二阶特征组合带来的新问题  
  1. 二阶特征组合会使得样本变得更稀疏. 因为二阶特征组合使得特征维度大大增加, 二样本数量还是原来那么多  
  2. 如果样本中, 没有某种二阶组合特征的样本, 则预测新样本是会不准确 (因为没有样本可以在那个参数上进行梯度下降)

#### 二. FM原理
1. 对于二阶特征组合的LR, 根据公式$\hat { y } =b+{ w }_{ 1 }x+\sum _{ i=1 }^{ n }{ \sum _{ j=i+1 }^{ n }{ { w }_{ ij }{ x }_{ i }{ x }_{ j } }  } $, 发现参数多出了最后面的${ w }_{ ij }$.   
 而上面的分析指出,训练样本可能不存在某些二阶特征组合对应的数据, 此时, 若我们能够找出每个属性$x_i$对应的权重$w_i$的隐因子向量$v_i$, 则2个隐因子向量的点积就能得到对应的二阶特征权重,即${ w }_{ ij }=\left< { v }_{ i },{ v }_{ j } \right> $. 从而有 :  $$\hat { y } =b+{ w }_{ 1 }x+\sum _{ i=1 }^{ n }{ \sum _{ j=i+1 }^{ n }{ \left< { v }_{ i },{ v }_{ j } \right> { x }_{ i }{ x }_{ j } }  } $$